# Linear cryptanalysis

In this file we study a method due to Matsui of breaking a block cipher with the help of linear relations. We analyze a simple cryptosystem described in the paper "A Tutorial on Linear and Differential Cryptanalysis" by Howard Heys.



## Cryptosystem (basic Substitution-Permutation network)

Input: plaintext of 16 bits length
Output: ciphertext of 16 bits length

Key: 80 bit key which we split into five 16-bit round keys $K_1, K_2, K_3, K_4, K_5$

This cryptosystem consists of 4 rounds. In each round $i$ we XOR with the round key $K_i$ split a block of 16 bits into 4 bit subblocks to which we apply an Sbox type transformation (non-linear). To each block we apply the same transformation. After that we apply to the list of $4$ blocks a permutation $P$. 
After the final round we XOR with the key $K_5$.

In [1]:
#Permutation P
Plist=[1,5,9,13,2,6,10,14,3,7,11,15,4,8,12,16];

def P(block16):
    return [block16[x] for x in Plist]

In [38]:
#SBox

def BlockToHex(key):
    size=len(key)//4
    hlist=[]
    for i in range(0,size):
        kpart=key[4*i:4*i+4]
        hlist.append(hex(int(''.join(map(str,kpart)),base=2))[2:].upper())
    return ''.join(hlist)

def leftpadding(li,n):
    return [0]*(n-len(li))+li

def HexToBlock(h):
    key=[]
    for hel in h:
        key+=leftpadding(map(int,bin(int(hel,base=16))[2:]),4)
    return key

Sboxlist=['E','4','D','1','2','F','B','8','3','A','6','C','5','9','0','7']

#print([x[1] for x in list(sorted([[x[1],x[0]] for x in zip(list(sorted(Sboxlist)),Sboxlist)]))])
Sboxinvlist=['E', '3', '4', '8', '1', 'C', 'A', 'F', '7', 'D', '9', '6', 'B', '2', '0', '5']

def SBox(chunk4):
    return HexToBlock(Sboxlist[int(BlockToHex(chunk4),base=16)])

def SBoxinv(chunk4):
    return HexToBlock(Sboxinvlist[int(BlockToHex(chunk4),base=16)])


def S(block16):
    Sout=[]
    for i in range(0,4):
        Sout+=SBox(block16[4*i:4*(i+1)])
    return Sout

def Sinv(block16):
    Sout=[]
    for i in range(0,4):
        Sout+=SBoxinv(block16[4*i:4*(i+1)])
    return Sout

In [2]:
#XORing lists
def ListXOR(li1,li2):
    return [li1[i]^^li2[i] for i in range(0,len(li1))]

In [25]:
#Key schedule
def KeySchedule(key80):
    return [key80[16*i:16*(i+1)] for i in range(0,5)]

def RoundKey(expkey,r):
    return expkey[r]

In [26]:
def SPNEncryption(plaintext16,key80):
    expkey=KeySchedule(key80)
    state=plaintext16
    
    #round 1
    state=ListXOR(state,RoundKey(expkey,0))
    state=S(state)
    state=P(state)
    
    #round 2
    
    state=ListXOR(state,RoundKey(expkey,1))
    state=S(state)
    state=P(state)
    
    #round 3
    
    state=ListXOR(state,RoundKey(expkey,2))
    state=S(state)
    state=P(state)
    
    #round 4
    
    state=ListXOR(state,RoundKey(expkey,3))
    state=S(state)
    
    #final XOR
    state=ListXOR(state,RoundKey(expkey,4))
    
    return state

In [ ]:
def SPNDecryption(plaintext16,key80):
    expkey=KeySchedule(key80)
    state=plaintext16
    
    pass




In [41]:
#Trying Sboxes

In [42]:
def Test1(X,Y):
    return SBox(X)==Y

In [43]:
def TestLinEq(X,Y,lin):
    a1,a2,a3,a4,b1,b2,b3,b4=list(lin)
    X1,X2,X3,X4=list(X)
    Y1,Y2,Y3,Y4=list(Y)
    return (a1*X1+a2*X2+a3*X3+a4*X4+b1*Y1+b2*Y2+b3*Y3+b4*Y4)%2==0

In [44]:
f=lambda X: TestLinEq(X,SBox(X),[0,1,1,0,1,0,1,1])

In [47]:
li=[[X1,X2,X3,X4] for X1 in [0,1] for X2 in [0,1] for X3 in [0,1] for X4 in [0,1] if f([X1,X2,X3,X4])]
li

[[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

In [50]:
bias=(len(li)/16)-1/2
bias

1/4